In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
from IPython.display import display
import os
import csv
from textblob import TextBlob

In [2]:
#Search term to be used on twitter scrape
search = "@subway"

#how to scrape
scraped_tweets= sntwitter.TwitterSearchScraper(search).get_items()
sliced_scraped_tweets = itertools.islice(scraped_tweets,10)

#create df of all the new data
df = pd.DataFrame(sliced_scraped_tweets)[['date', 'content']]

In [3]:
df

,date,content
0,2021-07-15 19:17:31+00:00,@SUBWAY ordered a steak &amp;cheese melt wit...
1,2021-07-15 19:16:27+00:00,Great idea @Subway at installing touchscreen m...
2,2021-07-15 19:16:24+00:00,So @SUBWAY is claining they can put whaever th...
3,2021-07-15 19:15:28+00:00,@HoaLy97197245 @stevis2point0 @mPinoe @S10Bird...
4,2021-07-15 19:14:23+00:00,@SUBWAY is a joke. I stop eating there when i ...
5,2021-07-15 19:09:15+00:00,@4lisaguerrero @SUBWAY Guess what I’m buying r...
6,2021-07-15 19:04:45+00:00,@SUBWAY https://t.co/8Og8Cm80Ro
7,2021-07-15 19:03:38+00:00,@WRadioColombia Señores de @Subway y @Sandwich...
8,2021-07-15 19:03:09+00:00,ayo @subway ur new wheat bread slaps
9,2021-07-15 19:02:37+00:00,@SUBWAY Why is there online ordering if they d...


In [4]:
#create array that will store the names of all the cities
city_array = []

#import dataframe containing populations of cities in california
california_cities_df = pd.read_csv('cal_populations_city.csv')


In [5]:
california_cities_df['pop_april_2010'].count()

455

In [6]:
california_cities_df.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765
1,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
3,Alameda,Albany,1908,15130.0,16327,16444,18539
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089


In [7]:
#only want to look at cities bigger than 75k to ensure there is enough data
bigger_ca_cities_df = california_cities_df[california_cities_df['pop_april_2010']>75000] 

#rslt_df = dataframe[dataframe['Percentage'] > 70] 

In [8]:
bigger_ca_cities_df.count()

#100 cities meet the criteria

County                 100
City                   100
Incorportation_date    100
pop_april_1980          95
pop_april_1990         100
pop_april_2000         100
pop_april_2010         100
dtype: int64

In [9]:
#df of bigger cities in CA
bigger_ca_cities_df.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089
7,Orange,Anaheim,1876,219494.0,266406,328014,336265
10,Contra Costa,Antioch,1872,42683.0,62195,90532,102372
24,Kern,Bakersfield,1873,105611.0,174820,246889,347483
25,Los Angeles,Baldwin Park,1956,50554.0,69330,75837,75390


In [10]:
city_array = bigger_ca_cities_df['City']

In [11]:
#import dataframe containing city coordinates
city_latlng = pd.read_csv('cal_cities_lat_long.csv')

In [12]:
#rename columns
city_latlng = city_latlng.rename(columns={'Name': 'City', 'Latitude': 'lat', 'Longitude': 'lng'})

city_latlng.head()

,City,lat,lng
0,Adelanto,34.582769,-117.409214
1,Agoura Hills,34.153339,-118.761675
2,Alameda,37.765206,-122.241636
3,Albany,37.886869,-122.297747
4,Alhambra,34.095286,-118.127014


In [13]:
#merge columns together, so have city populations, and long/lat
city_df_merged = bigger_ca_cities_df.merge(city_latlng, how='inner', on='City')

In [14]:
#
city_df_merged.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010,lat,lng
0,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089,34.095286,-118.127014
1,Orange,Anaheim,1876,219494.0,266406,328014,336265,33.835292,-117.914503
2,Contra Costa,Antioch,1872,42683.0,62195,90532,102372,38.004922,-121.805789
3,Kern,Bakersfield,1873,105611.0,174820,246889,347483,35.373292,-119.018711
4,Los Angeles,Baldwin Park,1956,50554.0,69330,75837,75390,34.085286,-117.960897


In [15]:
#create df of just columns that are needed
final_city_df = city_df_merged[['City', 'pop_april_2010', 'lat', 'lng']]

In [16]:
final_city_df.count()

City              100
pop_april_2010    100
lat               100
lng               100
dtype: int64

In [17]:
final_city_df = final_city_df.rename(columns={'City':'city'})

In [46]:
final_city_df

,city,pop_april_2010,lat,lng
0,Alhambra,83089,34.095286,-118.127014
1,Anaheim,336265,33.835292,-117.914503
2,Antioch,102372,38.004922,-121.805789
3,Bakersfield,347483,35.373292,-119.018711
4,Baldwin Park,75390,34.085286,-117.960897
...,...,...,...,...
95,Visalia,124442,36.330228,-119.292058
96,Vista,93834,33.200036,-117.242536
97,West Covina,106098,34.068622,-117.938953
98,Westminster,89701,33.751342,-117.993992


In [27]:
loc = f"{str(final_city_df.loc[33, 'lat'])}, {str(final_city_df.loc[33, 'lng'])}, 10km"

df_test = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    '@subway geocode:"{}"'.format(loc)).get_items(), 3))[['date', 'content', 'user']]

In [28]:
df_test

,date,content,user
0,2019-11-08 22:13:48+00:00,Dont go to @SUBWAY when its busy. They just fu...,"{'username': 'parsiak72', 'id': 1657803445, 'd..."


In [19]:
final_city_len = final_city_df['lat'].count()

In [38]:
##tests
#lat = "37.765206"
#lng = "-122.241636"


df_array = []
i = 0
#cycle through every 
#while i < final_city_df['lat'].count():
    
#testing
while i < final_city_len:
    #creates a string for twitter scrapper to read and use as location
    loc = f"{str(final_city_df.loc[i, 'lat'])}, {str(final_city_df.loc[i, 'lng'])}, 10km"
    
    print(i)
    df_test = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    '@subway geocode:"{}"'.format(loc)).get_items(), 5000))[['date', 'content', 'user']]
    print('scraper worked')
    if not df_test.empty:
        df_test['user_location'] = df_test['user'].apply(lambda x: x['location'])
        df_test['lat'] = final_city_df.loc[i, 'lat']
        df_test['lng'] = final_city_df.loc[i, 'lng']
        print('test')
        df_array.append(df_test)
    i = i +1

0
scraper worked
test
1
scraper worked
test
2
scraper worked
test
3
scraper worked
test
4
scraper worked
test
5
scraper worked
test
6
scraper worked
test
7
scraper worked
test
8
scraper worked
test
9
scraper worked
test
10
scraper worked
test
11
scraper worked
test
12
scraper worked
test
13
scraper worked
test
14
scraper worked
test
15
scraper worked
test
16
scraper worked
test
17
scraper worked
test
18
scraper worked
test
19
scraper worked
test
20
scraper worked
test
21
scraper worked
test
22
scraper worked
test
23
scraper worked
test
24
scraper worked
test
25
scraper worked
test
26
scraper worked
test
27
scraper worked
test
28
scraper worked
test
29
scraper worked
test
30
scraper worked
test
31
scraper worked
test
32
scraper worked
test
33
scraper worked
test
34
scraper worked
test
35
scraper worked
test
36
scraper worked
test
37
scraper worked
test
38
scraper worked
test
39
scraper worked
test
40
scraper worked
test
41
scraper worked
test
42
scraper worked
test
43
scraper worked
tes

KeyError: "None of [Index(['date', 'content', 'user'], dtype='object')] are in the [columns]"

In [43]:
i=92
while i < final_city_len:
    #creates a string for twitter scrapper to read and use as location
    loc = f"{str(final_city_df.loc[i, 'lat'])}, {str(final_city_df.loc[i, 'lng'])}, 10km"
    
    print(i)
    df_test = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    '@subway geocode:"{}"'.format(loc)).get_items(), 5000))[['date', 'content', 'user']]
    print('scraper worked')
    if not df_test.empty:
        df_test['user_location'] = df_test['user'].apply(lambda x: x['location'])
        df_test['lat'] = final_city_df.loc[i, 'lat']
        df_test['lng'] = final_city_df.loc[i, 'lng']
        print('test')
        df_array.append(df_test)
    i = i +1

92
scraper worked
test
93
scraper worked
test
94
scraper worked
test
95
scraper worked
test
96
scraper worked
test
97
scraper worked
test
98
scraper worked
test
99
scraper worked
test


In [44]:
for i in df_array:
    display(i)

,date,content,user,user_location,lat,lng
0,2021-07-15 17:45:02+00:00,@4lisaguerrero @SUBWAY I love tuna fish. I’ll ...,"{'username': 'ABConte', 'id': 69948869, 'displ...",Los Angeles,34.095286,-118.127014
1,2021-07-15 16:55:06+00:00,I have now done TWO investigations into the @S...,"{'username': '4lisaguerrero', 'id': 265705962,...","Los Angeles, CA",34.095286,-118.127014
2,2021-07-15 15:10:50+00:00,@roketeuss @SUBWAY 💀💀,"{'username': 'xyzeVfx', 'id': 1028245954531680...","Los Angeles, CA",34.095286,-118.127014
3,2021-07-15 02:54:48+00:00,🗣️Y'all still eating @SUBWAY after they told ...,"{'username': 'VersiVeEFAMG', 'id': 58457184, '...","Los Angeles, CA",34.095286,-118.127014
4,2021-07-14 21:42:57+00:00,Okay scientists: Is @Subway's claim true or j...,"{'username': 'Ms_Ellen_F', 'id': 286381811, 'd...",Los Angeles,34.095286,-118.127014
...,...,...,...,...,...,...
96,2017-03-03 20:45:47+00:00,"I'm at @Subway in Los Angeles, CA https://t.co...","{'username': 'MattySandz', 'id': 1442977914, '...","Burbank, CA",34.095286,-118.127014
97,2017-02-20 04:00:48+00:00,I ain't lying. @SUBWAY https://t.co/MuXHeZ5LeR,"{'username': 'J_LaSoul', 'id': 15878214, 'disp...","Los Angeles, CA",34.095286,-118.127014
98,2017-02-20 03:59:12+00:00,Damn. I was looking forward to eating my @SUBW...,"{'username': 'J_LaSoul', 'id': 15878214, 'disp...","Los Angeles, CA",34.095286,-118.127014
99,2017-02-09 05:40:30+00:00,"😄 (at @SUBWAY in El Monte, CA) https://t.co/a1...","{'username': 'ICameAllOver', 'id': 456867480, ...","El Monte, CA",34.095286,-118.127014


,date,content,user,user_location,lat,lng
0,2021-07-14 06:15:02+00:00,@SUBWAY These #coupons can't be used on the #C...,"{'username': 'ghosty4', 'id': 24838604, 'displ...","Anaheim, Ca",33.835292,-117.914503
1,2021-07-14 02:29:32+00:00,@NCSGI @SUBWAY Whoops missed this,"{'username': 'JizzelEtBass', 'id': 282252332, ...","Anaheim, CA",33.835292,-117.914503
2,2021-07-07 18:09:08+00:00,@RealLazyVulcan @SUBWAY @pizzahut @googlenews ...,"{'username': 'optimumsurf', 'id': 891873883099...","California, USA",33.835292,-117.914503
3,2021-07-07 18:03:16+00:00,@RealLazyVulcan @SUBWAY @pizzahut @googlenews ...,"{'username': 'optimumsurf', 'id': 891873883099...","California, USA",33.835292,-117.914503
4,2021-06-23 07:35:27+00:00,If your go to order at @SUBWAY is a #tuna sand...,"{'username': 'frankenbert_', 'id': 1913707124,...",,33.835292,-117.914503
...,...,...,...,...,...,...
336,2012-06-21 15:31:36+00:00,@SUBWAY then give us more on the sanwiches tha...,"{'username': 'stephberryumyum', 'id': 31200646...","Fullerton, CA",33.835292,-117.914503
337,2012-05-05 02:44:28+00:00,A lot of magic is happening right now. Eating ...,"{'username': 'LuiCalibre', 'id': 42300444, 'di...","New Orleans, LA",33.835292,-117.914503
338,2011-10-01 02:45:29+00:00,"Starting tomorrow, @Subway has codes for #unch...","{'username': 'johnnymorner', 'id': 107948035, ...","California, USA",33.835292,-117.914503
339,2011-05-11 00:25:55+00:00,Ok everybody so @justsimplyNANI eats the nasti...,"{'username': 'YoSoyELjefe_', 'id': 185115632, ...","Nuevo, CA",33.835292,-117.914503


,date,content,user,user_location,lat,lng
0,2021-02-23 00:40:49+00:00,@SUBWAY Protein Bowl was smacking 💪,"{'username': 'Close2God510', 'id': 32390765, '...","East Oakland, CA",38.004922,-121.805789
1,2019-01-11 04:14:12+00:00,@SUBWAY your oakley california workwrs are tra...,"{'username': 'tia__joness', 'id': 824854693138...",Bay Area,38.004922,-121.805789
2,2018-05-18 03:13:27+00:00,I am so relaxed after having @SUBWAY 😁,"{'username': 'sixgoldenlilies', 'id': 88804681...","Walnut Creek, CA",38.004922,-121.805789
3,2018-05-13 21:14:16+00:00,If we being honest @SUBWAY need to sponsor me ...,"{'username': 'Nu_Montana', 'id': 2337104166, '...","Based World, San Francisco🌁",38.004922,-121.805789
4,2017-03-24 00:29:29+00:00,I'd suck some toes for some @SUBWAY or @BWWing...,"{'username': 'Devin__Novak', 'id': 3146688980,...",Oahu,38.004922,-121.805789
5,2017-01-13 04:25:39+00:00,My first trip with my license was to @SUBWAY. ...,"{'username': 'Devin__Novak', 'id': 3146688980,...",Oahu,38.004922,-121.805789
6,2016-09-07 04:43:23+00:00,@Ylagan @Gxlindo @antonio__puente @SUBWAY @dom...,"{'username': 'iamkarinalive', 'id': 88480020, ...",Bay Area,38.004922,-121.805789
7,2016-09-07 04:34:36+00:00,@Gxlindo @antonio__puente @SUBWAY wow! What lo...,"{'username': 'iamkarinalive', 'id': 88480020, ...",Bay Area,38.004922,-121.805789
8,2016-08-13 18:19:55+00:00,Why Yall ain't got no more $5 footlong😐 @SUBWAY,"{'username': 'jovanni_12', 'id': 2826766756, '...",,38.004922,-121.805789
9,2016-03-19 19:50:14+00:00,"Lunch (at @Subway in Pittsburg, CA) https://t....","{'username': 'mszgrl72', 'id': 223055858, 'dis...","Pittsburg, CA",38.004922,-121.805789


,date,content,user,user_location,lat,lng
0,2021-07-13 14:20:14+00:00,@UberTieGuy @SUBWAY EXCUSE ME?!?!?!,"{'username': 'Tylerbagz', 'id': 78786457944745...","Bakersfield, CA",35.373292,-119.018711
1,2021-06-27 15:47:50+00:00,Amazing that @SUBWAY hasn’t been shut down yet...,"{'username': 'TheChiropractor', 'id': 84782427...",San Francisco,35.373292,-119.018711
2,2021-06-27 15:42:15+00:00,Is it Roe from the mutant fish @SUBWAY is usin...,"{'username': 'TheChiropractor', 'id': 84782427...",San Francisco,35.373292,-119.018711
3,2021-06-27 15:41:58+00:00,@sfchronicle is it Roe from the mutant fish @S...,"{'username': 'TheChiropractor', 'id': 84782427...",San Francisco,35.373292,-119.018711
4,2021-04-23 21:58:48+00:00,Getting a foot-long meatball marinara meal (at...,"{'username': 'kd6cae', 'id': 16707098, 'displa...","Palmdale, CA",35.373292,-119.018711
...,...,...,...,...,...,...
142,2013-06-13 00:41:27+00:00,@puja_Swag @SUBWAY Mmmmm meata-balla Sammywich!,"{'username': 'DawnaRieRie', 'id': 1265978408, ...","Bakersfield Calif,NOT Nevada!",35.373292,-119.018711
143,2013-05-17 22:01:03+00:00,"Time for some post-workout @SUBWAY, then hoppi...","{'username': 'driverRyanReed', 'id': 40747509,...",California / North Carolina,35.373292,-119.018711
144,2012-08-06 05:53:07+00:00,My host family brought me to @SUBWAY twice tod...,"{'username': 'MissFregaos', 'id': 70955246, 'd...",Wonju-si Wonju-no,35.373292,-119.018711
145,2012-07-18 22:07:53+00:00,@SUBWAY -- Who said that I was Jared and why? ...,"{'username': 'Cerbutar', 'id': 98202526, 'disp...","Bakersfield, CA 93301-4211 USA",35.373292,-119.018711


,date,content,user,user_location,lat,lng
0,2021-07-14 04:08:43+00:00,@SUBWAY Why does your tuna not have any traces...,"{'username': 'CrashSpyroBanjo', 'id': 12957275...","La Puente, CA",34.085286,-117.960897
1,2021-06-23 17:58:09+00:00,@RuthSch04593054 @SUBWAY No Tuna DNA in your f...,"{'username': 'nanorich', 'id': 20823753, 'disp...",Los Angeles,34.085286,-117.960897
2,2021-06-23 17:56:34+00:00,"@renderrain @MuckDNewsome88 @SUBWAY Maybe, may...","{'username': 'nanorich', 'id': 20823753, 'disp...",Los Angeles,34.085286,-117.960897
3,2021-06-23 17:53:53+00:00,"@SUBWAY Not Tuna, it seems.","{'username': 'nanorich', 'id': 20823753, 'disp...",Los Angeles,34.085286,-117.960897
4,2021-04-13 23:31:21+00:00,@SUBWAY why did u guys get rid of the 9 grain ...,"{'username': 'immatthew_', 'id': 2224369772, '...",,34.085286,-117.960897
...,...,...,...,...,...,...
180,2013-05-07 02:01:18+00:00,My family &amp; friends have switched to local...,"{'username': 'falvey_patrick', 'id': 527065482...",Los Angeles,34.085286,-117.960897
181,2013-04-30 20:38:24+00:00,Found some gift cards while cleaning my room. ...,"{'username': 'BigD923', 'id': 35308999, 'displ...","El Monte, CA",34.085286,-117.960897
182,2013-03-29 23:23:23+00:00,"Lunch time .. About to enjoy a footlong: ""Ital...","{'username': 'SANTOSALOFICIAL', 'id': 12814590...",United States,34.085286,-117.960897
183,2013-02-04 13:50:31+00:00,"@SUBWAY @SUBWAY, #SubwayShowdown and the appr...","{'username': 'ygalin', 'id': 76412707, 'displa...",ca,34.085286,-117.960897


...

,date,content,user,user_location,lat,lng
0,2019-12-04 04:43:25+00:00,@SUBWAY why don’t your PARTICIPATING locations...,"{'username': 'Savannahhhhaley', 'id': 10850317...","Porterville, CA",36.330228,-119.292058
1,2019-10-28 23:29:33+00:00,@SUBWAY LaLaLa remix October 30,"{'username': 'fckngdegenerate', 'id': 73535321...",Hell,36.330228,-119.292058
2,2019-03-11 19:40:43+00:00,@NereydaaSilvaa @alyssaru1z @SUBWAY 😌,"{'username': 'LeslieSilva__', 'id': 797611153,...","Visalia, CA",36.330228,-119.292058
3,2019-03-11 19:38:23+00:00,@alyssaru1z @SUBWAY @NereydaaSilvaa is shAKING,"{'username': 'LeslieSilva__', 'id': 797611153,...","Visalia, CA",36.330228,-119.292058
4,2016-12-12 01:16:22+00:00,@SUBWAY is sensational,"{'username': 'fernandothewoe', 'id': 310798997...",the gutter,36.330228,-119.292058
5,2016-12-05 07:18:09+00:00,My favorite cookie from @Subway is the #WhiteC...,"{'username': 'hecto2917', 'id': 4064067793, 'd...","Visalia, CA",36.330228,-119.292058
6,2016-10-26 02:52:07+00:00,What's the sub of the day @SUBWAY,"{'username': 'AleksEnglish', 'id': 57765215, '...","Sacramento, CA",36.330228,-119.292058
7,2015-07-08 16:01:28+00:00,Vasalia has everything but sadness. Can't wait...,"{'username': 'AlwaysRoxanne1', 'id': 116059546...","Los Angeles, CA",36.330228,-119.292058
8,2015-03-03 00:21:09+00:00,Visalia Cypress @Subway this salad is a bit sp...,"{'username': 'KC6YRU', 'id': 20338703, 'displa...","36.31981,-119.333387",36.330228,-119.292058
9,2014-09-14 20:26:23+00:00,I guess eating all that @Subway hasn't been do...,"{'username': 'StrokerBoob', 'id': 395720348, '...","California, USA",36.330228,-119.292058


,date,content,user,user_location,lat,lng
0,2020-10-18 23:03:12+00:00,@SUBWAY @MiamiDolphins Thanks Subway,"{'username': 'AustinT129', 'id': 1267141027808...",,33.200036,-117.242536
1,2020-05-30 22:13:29+00:00,@ZavierPedro @SUBWAY Nope,"{'username': 'harryburks4', 'id': 723574902679...","Vista, CA",33.200036,-117.242536
2,2020-05-30 20:12:32+00:00,@juaquin06264198 @SUBWAY Nah,"{'username': 'harryburks4', 'id': 723574902679...","Vista, CA",33.200036,-117.242536
3,2020-05-30 20:05:26+00:00,@ThanksBilly @SUBWAY Onions,"{'username': 'harryburks4', 'id': 723574902679...","Vista, CA",33.200036,-117.242536
4,2019-05-26 02:40:21+00:00,@SUBWAY thank you next.,"{'username': '_RickyLozano_', 'id': 2281279033...","San Diego, CA",33.200036,-117.242536
...,...,...,...,...,...,...
90,2013-09-22 23:58:12+00:00,#EatFresh #Tuna #Sandwich (at @Subway) [pic]: ...,"{'username': 'JoseMartinezSD', 'id': 553819218...","San Diego, CA",33.200036,-117.242536
91,2013-09-09 02:32:51+00:00,Dinner... #Subway #Melt #EatFresh (at @Subway)...,"{'username': 'JoseMartinezSD', 'id': 553819218...","San Diego, CA",33.200036,-117.242536
92,2013-09-08 19:30:17+00:00,"I'm at @Subway (Vista, CA) http://t.co/aiU6dsJscV","{'username': 'oceangirl6108', 'id': 18150059, ...",,33.200036,-117.242536
93,2013-08-12 00:39:48+00:00,Having to eat here since Mr. Sub won't deliver...,"{'username': '_KillerBeauty', 'id': 391283161,...",,33.200036,-117.242536


,date,content,user,user_location,lat,lng
0,2021-07-14 04:08:43+00:00,@SUBWAY Why does your tuna not have any traces...,"{'username': 'CrashSpyroBanjo', 'id': 12957275...","La Puente, CA",34.068622,-117.938953
1,2021-06-23 17:58:09+00:00,@RuthSch04593054 @SUBWAY No Tuna DNA in your f...,"{'username': 'nanorich', 'id': 20823753, 'disp...",Los Angeles,34.068622,-117.938953
2,2021-06-23 17:56:34+00:00,"@renderrain @MuckDNewsome88 @SUBWAY Maybe, may...","{'username': 'nanorich', 'id': 20823753, 'disp...",Los Angeles,34.068622,-117.938953
3,2021-06-23 17:53:53+00:00,"@SUBWAY Not Tuna, it seems.","{'username': 'nanorich', 'id': 20823753, 'disp...",Los Angeles,34.068622,-117.938953
4,2021-06-19 00:26:16+00:00,Another day another protein bowl from @SUBWAY ...,"{'username': 'JsAboutBusiness', 'id': 61721746...","Los Angeles, CA",34.068622,-117.938953
...,...,...,...,...,...,...
229,2013-06-17 17:39:35+00:00,@SUBWAY yummm!! Cucumbers!,"{'username': 'LAluvs27', 'id': 498068062, 'dis...",west covina ca,34.068622,-117.938953
230,2013-04-30 20:38:24+00:00,Found some gift cards while cleaning my room. ...,"{'username': 'BigD923', 'id': 35308999, 'displ...","El Monte, CA",34.068622,-117.938953
231,2013-03-29 23:23:23+00:00,"Lunch time .. About to enjoy a footlong: ""Ital...","{'username': 'SANTOSALOFICIAL', 'id': 12814590...",United States,34.068622,-117.938953
232,2013-02-04 13:50:31+00:00,"@SUBWAY @SUBWAY, #SubwayShowdown and the appr...","{'username': 'ygalin', 'id': 76412707, 'displa...",ca,34.068622,-117.938953


,date,content,user,user_location,lat,lng
0,2021-07-13 20:05:27+00:00,@willrowactor @SUBWAY I wish him a speedy reco...,"{'username': 'SandyRegion', 'id': 135865990454...",Midway City,33.751342,-117.993992
1,2021-07-10 21:00:33+00:00,MAJOR changes are coming to @SUBWAY. Read more...,"{'username': 'foodbeast', 'id': 17382260, 'dis...","Santa Ana, CA",33.751342,-117.993992
2,2021-03-16 19:47:45+00:00,@SUBWAY FOR LUNCH... TUNA WRAP ON SPINACH HA,"{'username': 'MIGGIE611', 'id': 23389244, 'dis...",California,33.751342,-117.993992
3,2020-11-15 23:25:34+00:00,"I'm at @Subway in Anaheim, CA https://t.co/vUF...","{'username': 'KBlack2595', 'id': 49882371, 'di...","Orange, CA",33.751342,-117.993992
4,2020-10-01 19:34:15+00:00,"Well, no more “foot longs” for me. Thanks, @S...","{'username': 'jonathan_shen7', 'id': 550933750...","Huntington Beach, California",33.751342,-117.993992
...,...,...,...,...,...,...
335,2011-10-03 21:19:41+00:00,Loving this @subway $5 anytober deal!!,"{'username': 'tommy1569', 'id': 165190180, 'di...",Nevada,33.751342,-117.993992
336,2011-10-03 21:16:45+00:00,Eating fresh Tweeps! @subway,"{'username': 'tommy1569', 'id': 165190180, 'di...",Nevada,33.751342,-117.993992
337,2011-10-01 02:45:29+00:00,"Starting tomorrow, @Subway has codes for #unch...","{'username': 'johnnymorner', 'id': 107948035, ...","California, USA",33.751342,-117.993992
338,2011-08-23 19:32:41+00:00,"@subway lunch, eating outdoors and listening t...","{'username': 'adanrico', 'id': 30963788, 'disp...","Bay Area, CA",33.751342,-117.993992


,date,content,user,user_location,lat,lng
0,2021-07-14 20:06:23+00:00,@SUBWAY How is the steak for your cheese steak...,"{'username': 'BrianCBock', 'id': 7310462, 'dis...","Downey, CA",33.979178,-118.032844
1,2021-07-14 04:08:43+00:00,@SUBWAY Why does your tuna not have any traces...,"{'username': 'CrashSpyroBanjo', 'id': 12957275...","La Puente, CA",33.979178,-118.032844
2,2021-03-09 02:42:01+00:00,"I'm at @SUBWAY in Los Angeles, CA https://t.co...","{'username': 'Insane4now', 'id': 34710996, 'di...",Los Angeles,33.979178,-118.032844
3,2021-02-25 17:50:03+00:00,Still waiting for @SUBWAY to do a campaign aro...,"{'username': 'NateoftheDead', 'id': 82796831, ...","Willoughby, July 1888",33.979178,-118.032844


In [45]:
print(loc)

33.979178, -118.032844, 10km


In [41]:
df_test = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    '@McDonalds geocode:"{}"'.format(loc)).get_items(), 1000))[['date', 'content']]

In [47]:
final_df_array = []

for df in df_array:
    print(df['date'].count())
    if df['date'].count() > 30:
        final_df_array.append(df)
        


101
341
17
147
185
47
118
367
9
75
406
16
28
218
123
35
8
2
228
1975
27
98
9
46
12
29
15
142
399
505
140
31
12
2
142
67
57
1356
334
385
33
21
202
761
16
12
21
3
3
133
52
255
56
177
367
12
37
10
189
172
40
89
99
56
49
3
176
15
62
17
2006
282
169
67
1
435
55
332
121
9
1173
18
2
8
54
193
57
19
298
6
290
4
3
36
12
95
234
340
4


In [48]:
for df in final_df_array:
    print(df['date'].count())

101
341
147
185
47
118
367
75
406
218
123
35
228
1975
98
46
142
399
505
140
31
142
67
57
1356
334
385
33
202
761
133
52
255
56
177
367
37
189
172
40
89
99
56
49
176
62
2006
282
169
67
435
55
332
121
1173
54
193
57
298
290
36
95
234
340


In [49]:
rows = []

for df in final_df_array:
    sentiment_total = 0
    df_len = df['date'].count()
    i = 0
    while i < df_len:
        testimonial = TextBlob(df.loc[i, 'content'])
        sentiment_total = sentiment_total + testimonial.sentiment.polarity
        sentiment_average = sentiment_total/df_len
        i = i+1
    print(f"({df.loc[1,'lat']}, {df.loc[1,'lng']}): Sentiment Total = {sentiment_total}, Average Sentiment: {sentiment_average}")
    rows.append([df.loc[1,'lat'],df.loc[1,'lng'], sentiment_total, sentiment_average])
    print("-" *42)

(34.095286, -118.127014): Sentiment Total = 7.718337166305917, Average Sentiment: 0.07641917986441502
------------------------------------------
(33.835292, -117.914503): Sentiment Total = 10.052303391053396, Average Sentiment: 0.029478895574936645
------------------------------------------
(35.373292, -119.018711): Sentiment Total = 7.318319213338746, Average Sentiment: 0.04978448444448126
------------------------------------------
(34.085286, -117.960897): Sentiment Total = 3.4696798340548347, Average Sentiment: 0.018755026130026133
------------------------------------------
(33.881683, -118.117011): Sentiment Total = 3.636805555555556, Average Sentiment: 0.07737884160756502
------------------------------------------
(37.871592, -122.272747): Sentiment Total = -0.9451319080225327, Average Sentiment: -0.00800959244086892
------------------------------------------
(33.867514, -117.998117): Sentiment Total = 18.58561958874459, Average Sentiment: 0.05064201522818689
---------------------

In [50]:
total_sentiment_df = pd.DataFrame(rows, columns=["lat", "lng", "sentiment_total", "sentiment_average"])

In [51]:
total_sentiment_df

,lat,lng,sentiment_total,sentiment_average
0,34.095286,-118.127014,7.718337,0.076419
1,33.835292,-117.914503,10.052303,0.029479
2,35.373292,-119.018711,7.318319,0.049784
3,34.085286,-117.960897,3.469680,0.018755
4,33.881683,-118.117011,3.636806,0.077379
...,...,...,...,...
59,33.745850,-117.826167,11.853365,0.040874
60,34.536108,-117.291158,2.960938,0.082248
61,33.200036,-117.242536,4.821019,0.050748
62,34.068622,-117.938953,13.042606,0.055738


In [52]:
total_sentiment_df.to_csv("subway_tweets_average_sentiments.csv")


In [54]:
# Specify the file to write to
output_path = os.path.join("..","tweet_csvs", "subway_tweets.csv")

# Open the file using "write" mode. Specify the variable to hold the contents
with open(output_path, 'w', newline='') as csvfile:
    
    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')
    
    csvwriter.writerow(['date', 'content', 'lat', 'lng', 'city', 'population', 'sentiment'])
    
    for df in final_df_array:
        print(df['date'].count())
        everything_bigger_final = pd.merge(df, final_city_df, how='inner', left_on=['lat', 'lng'], right_on=['lat', 'lng'])
        
        print(everything_bigger_final['date'].count())
        
        index_len = everything_bigger_final['date'].count()
        i = 0
        while i < index_len:
            testimonial = TextBlob(everything_bigger_final.loc[i, 'content'])
            
            csvwriter.writerow([everything_bigger_final.loc[i, 'date'], everything_bigger_final.loc[i, 'content'], everything_bigger_final.loc[i, 'lat'], everything_bigger_final.loc[i,'lng'], everything_bigger_final['city'][0], everything_bigger_final['pop_april_2010'][0], testimonial.sentiment.polarity])
            i = i+1
    # Write the first row (column headers)

    #date	content	user	user_location	lat	lng

101
101
341
341
147
147
185
185
47
47
118
118
367
367
75
75
406
406
218
218
123
123
35
35
228
228
1975
1975
98
98
46
46
142
142
399
399
505
505
140
140
31
31
142
142
67
67
57
57
1356
1356
334
334
385
385
33
33
202
202
761
761
133
133
52
52
255
255
56
56
177
177
367
367
37
37
189
189
172
172
40
40
89
89
99
99
56
56
49
49
176
176
62
62
2006
2006
282
282
169
169
67
67
435
435
55
55
332
332
121
121
1173
1173
54
54
193
193
57
57
298
298
290
290
36
36
95
95
234
234
340
340
